In [0]:
import mlflow

# 1. Use the Unity Catalog MLflow registry
mlflow.set_registry_uri("databricks-uc")

# 2. Setup
experiment_path = "/Shared/banking_credit_default_experiment"
model_name = "banking.default.banking_credit_default_model"

client = mlflow.tracking.MlflowClient()
exp = client.get_experiment_by_name(experiment_path)

# 3. Find the most recent successful run
runs = client.search_runs(
    experiment_ids=[exp.experiment_id],
    order_by=["attributes.start_time DESC"],
    max_results=1
)

if not runs:
    raise Exception("❌ No MLflow runs found. Run Step 3 first.")

best_run = runs[0]
best_run_id = best_run.info.run_id
print(f"🏁 Latest run found: {best_run_id}")

# 4. Register the model artifact from that run
model_uri = f"runs:/{best_run_id}/model"

# Register the model in Unity Catalog
result = mlflow.register_model(
    model_uri=model_uri,
    name=model_name
)
model_version = result.version

print(f"✅ Model {model_name} v{model_version} registered successfully in Unity Catalog registry")

# 5. Assign the "champion" alias to the new model version
client.set_registered_model_alias(
    name=model_name,
    alias="champion",
    version=model_version
)
print(f"🏷️ Assigned 'champion' alias to {model_name} v{model_version}")

# 6. Log metadata (optional) in a nested run to avoid parameter key collision
with mlflow.start_run(run_id=best_run_id):
    with mlflow.start_run(nested=True):
        mlflow.log_param("registered_model_name", model_name)
        mlflow.log_param("registered_model_version", model_version)
        print("ℹ️ Logged registration metadata to a nested run.")

print("✅ Step 4 complete – model registered in UC registry and 'champion' alias assigned.")